In [91]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['mnist_train.csv', 'mnist_test.csv']


In [92]:
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import clear_output
import pandas as pd
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

In [93]:
train = pd.read_csv("../input/mnist_train.csv")
test = pd.read_csv("../input/mnist_test.csv")

In [94]:
print(train.shape)
print(test.shape)
print(train.columns)
print(train[:2])
print(test[:2])
print(train[:10]["label"])
print(test[:10]["label"])

(60000, 785)
(10000, 785)
Index(['label', '1x1', '1x2', '1x3', '1x4', '1x5', '1x6', '1x7', '1x8', '1x9',
       ...
       '28x19', '28x20', '28x21', '28x22', '28x23', '28x24', '28x25', '28x26',
       '28x27', '28x28'],
      dtype='object', length=785)
   label  1x1  1x2  1x3  1x4  ...    28x24  28x25  28x26  28x27  28x28
0      5    0    0    0    0  ...        0      0      0      0      0
1      0    0    0    0    0  ...        0      0      0      0      0

[2 rows x 785 columns]
   label  1x1  1x2  1x3  1x4  ...    28x24  28x25  28x26  28x27  28x28
0      7    0    0    0    0  ...        0      0      0      0      0
1      2    0    0    0    0  ...        0      0      0      0      0

[2 rows x 785 columns]
0    5
1    0
2    4
3    1
4    9
5    2
6    1
7    3
8    1
9    4
Name: label, dtype: int64
0    7
1    2
2    1
3    0
4    4
5    1
6    4
7    9
8    5
9    9
Name: label, dtype: int64


In [95]:
train = np.array(train)
test = np.array(test)

In [96]:
print(train[10:15])
print(test[10:15])

[[3 0 0 ... 0 0 0]
 [5 0 0 ... 0 0 0]
 [3 0 0 ... 0 0 0]
 [6 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [6 0 0 ... 0 0 0]
 [9 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [97]:
train_x = train[:,1:]
train_y = pd.get_dummies(train[:,0])
test_x = test[:,1:]
test_y = pd.get_dummies(test[:,0])

In [98]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)
print(train_x[10:12])
print(train_y[10:12])
print(test_x[10:12])
print(test_y[10:12])

(60000, 784)
(60000, 10)
(10000, 784)
(10000, 10)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
    0  1  2  3  4  5  6  7  8  9
10  0  0  0  1  0  0  0  0  0  0
11  0  0  0  0  0  1  0  0  0  0
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
    0  1  2  3  4  5  6  7  8  9
10  1  0  0  0  0  0  0  0  0  0
11  0  0  0  0  0  0  1  0  0  0


#### We will treat each image as a sequence of 28 rows of 28 pixels each (since each MNIST image is 28 × 28 pixels). We will use cells of 150 recurrent neurons, plus a fully connected layer containing 10 neurons (one per class) connected to the output of the last time step, followed by a softmax layer

In [99]:
#NETWORK PARAMETERS
n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

In [100]:
train_x = train_x.reshape(-1,n_steps,n_inputs)
test_x = test_x.reshape(-1,n_steps,n_inputs)

In [101]:
print(train_x.shape)
print(test_x.shape)
print(train_x[0:1])
print(test_x[0:1])

(60000, 28, 28)
(10000, 28, 28)
[[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
     0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
     0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
     0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
     0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
     0   0   0   0   0   0   0   0   0   0   0]
  [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126
   136 175  26 166 255 247 127   0   0   0   0]
  [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253
   253 225 172 253 242 195  64   0   0   0   0]
  [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253
   251  93  82  82  56  39   0   0   0   0   0]
  [  0   0   0  

In [102]:
train_X , train_y = shuffle(train_x , train_y)
test_X , test_y = shuffle(test_x , test_y)

In [103]:
tf.reset_default_graph()

In [104]:
# Training Parameters
learning_rate = 0.001
training_iters = 500
batch_size = 150
display_step = 200

In [105]:
X = tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y = tf.placeholder(tf.int32,[None,n_outputs])

In [106]:
def RNN(X):
    basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
    outputs , states = tf.nn.dynamic_rnn(basic_cell,X,dtype=tf.float32)
    out = tf.layers.dense(states, n_outputs)
    out = tf.nn.softmax(out)
    return out
    

In [107]:
prediction = RNN(X)

In [108]:
# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

In [109]:
# correct_pred = tf.nn.in_top_k(prediction, y, 1)

In [110]:
# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [111]:
sess = tf.Session()
sess.run(init) 

for i in range(training_iters):
    for batch in range(len(train_X)//batch_size):
        batch_x = train_X[batch*batch_size:min((batch+1)*batch_size,len(train_X))]
        batch_y = train_y[batch*batch_size:min((batch+1)*batch_size,len(train_y))]
        batch_x = batch_x.reshape((-1, n_steps, n_inputs))

        sess.run(train_op, feed_dict={X: batch_x, y: batch_y})
        loss = sess.run([loss_op], feed_dict={X: batch_x, y: batch_y})
    predTest = sess.run(prediction , feed_dict={X:test_X})
    acc_train = accuracy.eval(session=sess, feed_dict={X: batch_x, y: batch_y})
    acc_test = accuracy.eval(session=sess, feed_dict={X: test_X, y: test_y})
    print("Iter "+str(i)+" Out of",training_iters , " Loss= ",loss,"Train accuracy:", acc_train, "Test accuracy:", acc_test)
        
        
            # Calculate batch loss and accuracy
#         loss = sess.run([loss_op], feed_dict={X: batch_x, y: batch_y})
    
#     predTest = sess.run(prediction , feed_dict={X:test_X})

#     p = np.argmax(predTest,1)
#     t = np.argmax(np.array(test_y),1)

#     acc = accuracy_score(p,t)
#     print("Iter "+str(i)+" Out of",training_iters , " Loss= ",loss, "acc=",acc )
            
#     acc = sess.run([accuracy], feed_dict={X: batch_x, y: batch_y})
        
#     print("Step " + str(i) + ",        Batch Loss= ",loss, ",       Training Accuracy= ",acc)
    
print("Optimization Finished!")

Iter 0 Out of 500  Loss=  [1.8679792] Train accuracy: 0.6 Test accuracy: 0.5102
Iter 1 Out of 500  Loss=  [1.8815346] Train accuracy: 0.58 Test accuracy: 0.5102
Iter 2 Out of 500  Loss=  [1.8508142] Train accuracy: 0.62666667 Test accuracy: 0.5201
Iter 3 Out of 500  Loss=  [1.8966292] Train accuracy: 0.56666666 Test accuracy: 0.5166
Iter 4 Out of 500  Loss=  [1.8804394] Train accuracy: 0.5733333 Test accuracy: 0.5219
Iter 5 Out of 500  Loss=  [1.8794837] Train accuracy: 0.58 Test accuracy: 0.5207
Iter 6 Out of 500  Loss=  [1.8815492] Train accuracy: 0.58666664 Test accuracy: 0.5225
Iter 7 Out of 500  Loss=  [1.8756919] Train accuracy: 0.58 Test accuracy: 0.523
Iter 8 Out of 500  Loss=  [1.8723665] Train accuracy: 0.5733333 Test accuracy: 0.5264
Iter 9 Out of 500  Loss=  [1.8714194] Train accuracy: 0.58666664 Test accuracy: 0.5265
Iter 10 Out of 500  Loss=  [1.8988777] Train accuracy: 0.55333334 Test accuracy: 0.525
Iter 11 Out of 500  Loss=  [1.8906972] Train accuracy: 0.5733333 Test a

Iter 97 Out of 500  Loss=  [1.8787277] Train accuracy: 0.58 Test accuracy: 0.5438
Iter 98 Out of 500  Loss=  [1.8831667] Train accuracy: 0.58 Test accuracy: 0.5485
Iter 99 Out of 500  Loss=  [1.8703184] Train accuracy: 0.5933333 Test accuracy: 0.5467
Iter 100 Out of 500  Loss=  [1.8397363] Train accuracy: 0.61333334 Test accuracy: 0.5543
Iter 101 Out of 500  Loss=  [1.8544335] Train accuracy: 0.5933333 Test accuracy: 0.5501
Iter 102 Out of 500  Loss=  [1.8759408] Train accuracy: 0.58666664 Test accuracy: 0.5443
Iter 103 Out of 500  Loss=  [1.8637317] Train accuracy: 0.58666664 Test accuracy: 0.5484
Iter 104 Out of 500  Loss=  [1.8892639] Train accuracy: 0.5733333 Test accuracy: 0.5505
Iter 105 Out of 500  Loss=  [1.8707778] Train accuracy: 0.58666664 Test accuracy: 0.554
Iter 106 Out of 500  Loss=  [1.8627604] Train accuracy: 0.6066667 Test accuracy: 0.5528
Iter 107 Out of 500  Loss=  [1.8266042] Train accuracy: 0.64 Test accuracy: 0.5558
Iter 108 Out of 500  Loss=  [1.8322374] Train a

Iter 192 Out of 500  Loss=  [1.8543302] Train accuracy: 0.61333334 Test accuracy: 0.5513
Iter 193 Out of 500  Loss=  [1.8597859] Train accuracy: 0.6 Test accuracy: 0.5501
Iter 194 Out of 500  Loss=  [1.8459989] Train accuracy: 0.6066667 Test accuracy: 0.5558
Iter 195 Out of 500  Loss=  [1.8192343] Train accuracy: 0.64666665 Test accuracy: 0.5499
Iter 196 Out of 500  Loss=  [1.8418498] Train accuracy: 0.61333334 Test accuracy: 0.5579
Iter 197 Out of 500  Loss=  [1.8388427] Train accuracy: 0.62666667 Test accuracy: 0.554
Iter 198 Out of 500  Loss=  [1.809319] Train accuracy: 0.6666667 Test accuracy: 0.5543
Iter 199 Out of 500  Loss=  [1.8461703] Train accuracy: 0.6066667 Test accuracy: 0.5527
Iter 200 Out of 500  Loss=  [1.8573799] Train accuracy: 0.6 Test accuracy: 0.5518
Iter 201 Out of 500  Loss=  [1.810335] Train accuracy: 0.66 Test accuracy: 0.5561
Iter 202 Out of 500  Loss=  [1.8741375] Train accuracy: 0.58 Test accuracy: 0.5502
Iter 203 Out of 500  Loss=  [1.8496425] Train accurac

Iter 288 Out of 500  Loss=  [1.8365827] Train accuracy: 0.62 Test accuracy: 0.563
Iter 289 Out of 500  Loss=  [1.8625284] Train accuracy: 0.5933333 Test accuracy: 0.5491
Iter 290 Out of 500  Loss=  [1.8888221] Train accuracy: 0.5733333 Test accuracy: 0.552
Iter 291 Out of 500  Loss=  [1.8707218] Train accuracy: 0.5933333 Test accuracy: 0.5506
Iter 292 Out of 500  Loss=  [1.8447982] Train accuracy: 0.62 Test accuracy: 0.5562
Iter 293 Out of 500  Loss=  [1.8529824] Train accuracy: 0.6066667 Test accuracy: 0.5557
Iter 294 Out of 500  Loss=  [1.8532418] Train accuracy: 0.6066667 Test accuracy: 0.5579
Iter 295 Out of 500  Loss=  [1.8590287] Train accuracy: 0.6 Test accuracy: 0.5571
Iter 296 Out of 500  Loss=  [1.8663564] Train accuracy: 0.5933333 Test accuracy: 0.5444
Iter 297 Out of 500  Loss=  [1.8470122] Train accuracy: 0.62 Test accuracy: 0.5593
Iter 298 Out of 500  Loss=  [1.8269979] Train accuracy: 0.64666665 Test accuracy: 0.5525
Iter 299 Out of 500  Loss=  [1.8543667] Train accuracy

Iter 383 Out of 500  Loss=  [1.8221459] Train accuracy: 0.64666665 Test accuracy: 0.5523
Iter 384 Out of 500  Loss=  [1.8232015] Train accuracy: 0.6333333 Test accuracy: 0.5574
Iter 385 Out of 500  Loss=  [1.8688607] Train accuracy: 0.5933333 Test accuracy: 0.5517
Iter 386 Out of 500  Loss=  [1.8608185] Train accuracy: 0.6066667 Test accuracy: 0.559
Iter 387 Out of 500  Loss=  [1.8319275] Train accuracy: 0.62666667 Test accuracy: 0.5558
Iter 388 Out of 500  Loss=  [1.8688363] Train accuracy: 0.58666664 Test accuracy: 0.5538
Iter 389 Out of 500  Loss=  [1.8196673] Train accuracy: 0.6533333 Test accuracy: 0.5551
Iter 390 Out of 500  Loss=  [1.8365242] Train accuracy: 0.6333333 Test accuracy: 0.5598
Iter 391 Out of 500  Loss=  [1.8659803] Train accuracy: 0.6066667 Test accuracy: 0.5571
Iter 392 Out of 500  Loss=  [1.8583183] Train accuracy: 0.6 Test accuracy: 0.5528
Iter 393 Out of 500  Loss=  [1.8104286] Train accuracy: 0.64666665 Test accuracy: 0.55
Iter 394 Out of 500  Loss=  [1.847286

Iter 479 Out of 500  Loss=  [1.8406632] Train accuracy: 0.62666667 Test accuracy: 0.556
Iter 480 Out of 500  Loss=  [1.8373958] Train accuracy: 0.62666667 Test accuracy: 0.5567
Iter 481 Out of 500  Loss=  [1.8739288] Train accuracy: 0.58 Test accuracy: 0.5568
Iter 482 Out of 500  Loss=  [1.8640198] Train accuracy: 0.6 Test accuracy: 0.5568
Iter 483 Out of 500  Loss=  [1.8703164] Train accuracy: 0.58 Test accuracy: 0.5531
Iter 484 Out of 500  Loss=  [1.877233] Train accuracy: 0.58 Test accuracy: 0.5542
Iter 485 Out of 500  Loss=  [1.8492668] Train accuracy: 0.5933333 Test accuracy: 0.5591
Iter 486 Out of 500  Loss=  [1.8477354] Train accuracy: 0.6066667 Test accuracy: 0.5605
Iter 487 Out of 500  Loss=  [1.8476491] Train accuracy: 0.61333334 Test accuracy: 0.5603
Iter 488 Out of 500  Loss=  [1.8467396] Train accuracy: 0.61333334 Test accuracy: 0.5639
Iter 489 Out of 500  Loss=  [1.8623458] Train accuracy: 0.5933333 Test accuracy: 0.5594
Iter 490 Out of 500  Loss=  [1.8276699] Train accur

In [113]:
while(True):
    r = np.random.randint(9000)
    test_img = np.reshape(test_X[r], (28,28))
    plt.imshow(test_img, cmap="gray")
    test_pred = sess.run(prediction, feed_dict = {X:[test_X[r]]})
    print("Model : I think it is :    ",np.argmax(test_pred))
    plt.show()
    
    if input("Enter n to exit")=='n':
        break
clear_output();

In [114]:
wrong = test_X[tf.argmax(prediction, 1)!=tf.argmax(y, 1)]
wrong.shape

(1, 10000, 28, 28)

In [115]:
a,b,c, d = wrong.shape

In [116]:
wrong = np.reshape(wrong, (b,c,d))   
wrong.shape

(10000, 28, 28)

In [117]:
while(True):
    r=np.random.randint(b)
    plt.imshow(wrong[r].reshape((28,28)),cmap="gray")
    test_pred_1=sess.run(prediction, feed_dict = {X:[wrong[r]]})
    print("Model : I think it is :    ",np.argmax(test_pred_1))
    plt.show()
    
    if input("Enter n to exit")=='n':
        break
clear_output();

In [118]:
p = np.argmax(predTest,1)
print(p)
t = np.argmax(np.array(test_y),1)
print(t)
acc = accuracy_score(p,t)
print(acc*100)

[5 0 1 ... 5 8 8]
[8 5 1 ... 5 8 3]
55.47


In [119]:
print("Saving Weights")
saver = tf.train.Saver()
saver.save(sess,"weights_"+str(i)+"/weights.ckpt")
print("Weights Saved")

Saving Weights
Weights Saved
